In [42]:
from abc import ABC, abstractmethod
import warnings
import random
import math
from copy import deepcopy

from numpy.f2py.auxfuncs import throw_error

In [449]:
class ActivationFunction(ABC):

    @abstractmethod
    def __call__(self, x):
        raise Exception("Not implemented call method in activation function.")


    def ddx(self, x):
        raise Exception("Not implemented d/dx method in activation function.")

class ReLu(ActivationFunction):
    def __call__(self, x : float):
        return max(0 ,x)
    def ddx(self, x):
        return 1 if x>=0 else 0



In [450]:
class ErrorFunction(ABC):
    @abstractmethod
    def __call__(self, *args, **kwargs):
        pass
    @abstractmethod
    def ddx(self):
        pass

class MSE(ErrorFunction):
    def __call__(self, y_pred: list, y_true : list):
        pass

In [520]:
class Layer(ABC):
    @abstractmethod
    def __call__(self, x):
        pass

    # def deriff(self, x):
    #     pass
    #
    #
    # def ddw_i(self):
    #     raise Exception("Nie zaimplementowano, d/dw_i")
    #
    #
    # def ddb(self):
    #     raise Exception("Nie zaimplementowano d/db")


class Perceptron(Layer):

    def __init__(self, input_size, initialization_type : str = "he"):
        self.weight = [random.gauss(mu = 0, sigma=math.sqrt(2/input_size)) for i in range(input_size)]
        self.bias = random.gauss(mu = 0, sigma=0.1)

        self.grad_weight = [0 for i in range(len(self.weight)) ]
        self.grad_bias = 0
        self.backward_grad = [0 for i in range(len(self.weight)) ] # grad z poprzedniego

        self.batch_grad_weight = [0 for i in range(len(self.weight)) ]
        self.batch_grad_bias = 0

        self.forward_value = None
        self.forward_x = None


        self.weight = [2 for i in range(input_size)]
        self.bias = 2
        # self.activate_function = ReLu()



    def __call__(self, x):

        if len(x) != len(self.weight):
            raise Exception("Size od weights not match input size.")
        output = [ self.weight[i] * x[i] for i in range(len(x)) ]
        output.append(self.bias)
        return sum(output)

    def forward(self, x):
        self.forward_value = self(x)
        self.forward_x = x
        return self.forward_value



    def zero_batch_grad(self):
        self.batch_grad_weight = [0 for i in range(len(self.weight)) ]
        self.batch_grad_bias = 0
    def zero_grad(self):
        self.grad_weight = [0 for i in range(len(self.weight)) ]
        self.grad_bias = 0
        self.backward_grad = [0 for i in range(len(self.weight)) ] # grad z poprzedniego



    def ddw_i(self, i):
        return self.forward_x[i]
    def ddx_i(self,i):
        return self.weight[i]

    def ddw(self, previous_grad): # x w tym przypadku jest previous_grad
        for i in range (len(self.grad_weight)):
            self.grad_weight[i] += self.ddw_i(i)*previous_grad
    def ddb(self, previous_grad):
        self.grad_bias += previous_grad
    def calculate_backward_grad(self, previous_grad):
        self.backward_grad = ([self.ddx_i(i)*previous_grad  for i in range(len(self.grad_weight))])



    def add_grad_to_batch_grad(self):
        self.batch_grad_bias += self.grad_bias
        for i in range(len(self.grad_weight)):
            self.batch_grad_weight[i] += self.grad_weight[i]
    def normalize_batch_grad(self, batch_size : int):
        self.batch_grad_bias /= batch_size
        for i in range(len(self.grad_weight)):
            self.batch_grad_weight[i] /= batch_size





class Linear(Layer):

    def __init__(self,input_size : int, output_size : int, activation_function : str):

        self.activation_function = None
        self.layer = []



        match activation_function:
            case "ReLu":
                self.activation_function = ReLu()
                self.layer = [Perceptron(input_size = input_size) for i in range(output_size)]
            # case "Sigmoid":
            #     self.activation_function = ReLu()
            #     self.layers = [Perceptron() for i in range(output_size)]
            # case "Tanh":
            #     self.activation_function = ReLu()
            #     self.layers = [Perceptron() for i in range(output_size)]
            # case "LeakyReLu":
            #     self.activation_function = ReLu()
            #     self.layers = [Perceptron() for i in range(output_size)]
            case _ :
                warnings.warn(f"{activation_function} is not a activation function.")
                self.activation_function = lambda x: x
                self.layer = [Perceptron(input_size=input_size) for i in range(output_size)]

    def __call__(self, inputs : list):
        outputs = [perceptron(inputs) for perceptron in self.layer]
        return [self.activation_function(output) for output in outputs]

    def forward(self, inputs):
        outputs = [perceptron.forward(inputs) for perceptron in self.layer]
        return [self.activation_function(output) for output in outputs]


    def zero_batch_grad(self):
        for perceptron in self.layer:
            perceptron.zero_batch_grad()
    def zero_grad(self):
        for perceptron in self.layer:
            perceptron.zero_grad()
    def add_grad_to_batch_grad(self):
        for perceptron in self.layer:
            perceptron.add_grad_to_batch_grad()
    def normalize_batch_grad(self, batch_size : int):
        for perceptron in self.layer:
            perceptron.normalize_batch_grad(batch_size)


    # def ddx_i(self, i:int):
    #     return [ perceptron.ddx_i(i) for perceptron in self.layer]
    # def d_activation_dx_i(self, i):
    #     return self.activation_function.ddx(self.layer[i].value)



    def print_forward(self):
        print([perceptron.forward_value for perceptron in self.layer])
    def print_grad(self):
        print([[perceptron.grad_weight, perceptron.grad_bias ]for perceptron in self.layer])
    def print_backward_grad(self):
        print([perceptron.backward_grad for perceptron in self.layer])

    def print_batch_grad(self):
        print([[perceptron.batch_grad_weight, perceptron.batch_grad_bias ]for perceptron in self.layer])



In [523]:
class Model:
    def __init__(self):
        self.layers = [Linear(2,2,"ReLu"),
                      Linear(2,2, "ReLu"),
                      Linear(2,1, "ReLu")]
    def __call__(self, x_original : list):
        x = deepcopy(x_original)
        for layer in self.layers:
            x = layer(x)
        return x

    def forward(self, x_original):
        x = deepcopy(x_original)
        for layer in self.layers:
            x = layer.forward(x)
        return x


    def zero_batch_grad(self):
        for layer in self.layers:
            layer.zero_batch_grad()
    def zero_grad(self):
        for layer in self.layers:
            layer.zero_grad()
    def add_grad_to_batch_grad(self):
        for layer in self.layers:
            layer.add_grad_to_batch_grad()
    def normalize_batch_grad(self, batch_size : int):
        for layer in self.layers:
            layer.normalize_batch_grad(batch_size)


    def backward(self):
        self.zero_grad()

        for perceptron in self.layers[len(self.layers)-1].layer:
            x = perceptron.forward_value
            previous_grad = self.layers[len(self.layers)-1].activation_function.ddx(x)
            perceptron.ddb(previous_grad)
            perceptron.ddw(previous_grad )
            perceptron.calculate_backward_grad( previous_grad)

        self.layers.reverse()
        try:
            for i in range(1, len(self.layers)):
                linear = self.layers[i]
                previous_linear = self.layers[i-1]
                for j in range(len(linear.layer)):
                    perceptron = linear.layer[j]
                    for k in range(len(previous_linear.layer)):
                        previous_perceptron = previous_linear.layer[k]
                        previous_grad = previous_perceptron.backward_grad[j]
                        # previous_grad *= d_a
                        # print(previous_grad)
                        # 1/0

                        perceptron.ddb(previous_grad )
                        perceptron.ddw(previous_grad ) # TODO: domnóż to przez pochodną f aktywacji
                        perceptron.calculate_backward_grad( previous_grad )

        except Exception as e:
            self.layers.reverse()
            raise e

        self.layers.reverse()



    def print_values(self, type:str):
        self.layers.reverse()
        try:
            match type:
                case "backward_grad":
                    for layer in self.layers:
                        print("Backward_grad")
                        layer.print_backward_grad()

                case "grad":
                    for layer in self.layers:
                        print("Grads:")
                        layer.print_grad()

                case "forward":
                    for layer in self.layers:
                        print("Forward")
                        layer.print_forward()

                case "batch":
                    for layer in self.layers:
                        print("Batch_rads:")
                        layer.print_batch_grad()
                case "all":

                    print("Backward_grad")
                    for layer in self.layers:
                        layer.print_backward_grad()
                    print("Grads:")
                    for layer in self.layers:
                        layer.print_grad()

                    print("Batch_grad:")
                    for layer in self.layers:
                        layer.print_batch_grad()

                    print("Forward")
                    for layer in self.layers:
                        layer.print_forward()
                case _:
                    print(f"{type} is not correct type of print.")


        except Exception as e:
            self.layers.reverse()
            raise e

        self.layers.reverse()






model = Model()

In [524]:
print(model([1,2]))
print(model.forward([1,2]))
model.backward()

[138]
[138]


In [525]:
model.add_grad_to_batch_grad()
model.add_grad_to_batch_grad()
model.add_grad_to_batch_grad()
model.normalize_batch_grad(3)

In [526]:
model.print_values("all")

Backward_grad
[[2, 2]]
[[4, 4], [4, 4]]
[[8, 8], [8, 8]]
Grads:
[[[34, 34], 1]]
[[[16, 16], 2], [[16, 16], 2]]
[[[8, 16], 8], [[8, 16], 8]]
Batch_grad:
[[[34.0, 34.0], 1.0]]
[[[16.0, 16.0], 2.0], [[16.0, 16.0], 2.0]]
[[[8.0, 16.0], 8.0], [[8.0, 16.0], 8.0]]
Forward
[138]
[34, 34]
[8, 8]


In [476]:
for i in range(len(model.layers)):
    number_layer = i
    print(i,model.layers[number_layer].layer[0].backward_grad, model.layers[number_layer].layer[0].grad_weight, model.layers[number_layer].layer[0].grad_bias)

0 [8, 8] [8, 16] 8
1 [4, 4] [16, 16] 2
2 [2, 2] [34, 34] 1


In [104]:
a = Perceptron(2)
a([1,2])

-0.188139308930177

In [105]:
# a.forward([1,2])
# print(a.value)

-0.188139308930177


In [36]:
a.bias

-0.08249214176724436

In [39]:
a = None
warnings.warn(f"its {a}")

/var/folders/44/w406vwt16zl0wwv25rzht4fc0000gn/T/ipykernel_68565/3845773530.py:2: UserWarning: its None
  warnings.warn(f"its {a}")


In [205]:
print([1,2].reverse())

None
